In [2]:
!pip install pulp -i https://pypi.tuna.tsinghua.edu.cn/simple/

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
     |████████████████████████████████| 40.6 MB 94 kB/s eta 0:00:011    |█████▊                          | 7.3 MB 48 kB/s eta 0:11:21     |██████                          | 7.6 MB 858 kB/s eta 0:00:39     |████████▌                       | 10.7 MB 1.6 MB/s eta 0:00:20     |████████████████████▎           | 25.7 MB 905 kB/s eta 0:00:17     |█████████████████████▏          | 26.9 MB 99 kB/s eta 0:02:18


In [3]:
!pip install pprint -i https://pypi.tuna.tsinghua.edu.cn/simple/

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
  Created wheel for pprint: filename=pprint-0.1-py3-none-any.whl size=1250 sha256=3668c0ceb06841c3b343f390cb34e686f8958c64b0fb6e5e5e0b078fd66c0112
  Stored in directory: /Users/yangkaixiang/Library/Caches/pip/wheels/c4/d7/d8/8c4c4efc441cff850497342bfe37a54b3101593e503b8ef225
Successfully built pprint


In [4]:
import pulp
import numpy as np
from pprint import pprint

# 定义问题
def farming_program(costs, x_max, y_max):
    row = len(costs)
    col = len(costs[0])
    print(f'row:{row} col:{col}')
    # 定义问题
    prob = pulp.LpProblem('Transportation', sense=pulp.LpMaximize)
    # 定义决策变量
    var = [[pulp.LpVariable(f'x{i}{j}', lowBound=0, cat=pulp.LpInteger) for j in range(col)] for i in range(row)] # 先执行外层list 再执行内层list
    # 递归展开列表
    flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]
    print('var=', var)
    print('flatten=', flatten)
    print('var[1]', var[1])
    # 定义目标函数 （单价* {ij}运量）
    prob += pulp.lpDot(flatten(var), costs.flatten())
    # 定义约束条件
    for i in range(row):
        prob += pulp.lpSum(var[i]) <= x_max[i]
    for j in range(col):
        prob += pulp.lpSum([var[i][j] for i in range(row)]) <= y_max[j]

    prob.solve()
    result = {'objective':pulp.value(prob.objective), \
              'var': [[pulp.value(var[i][j]) for j in range(col)] for i in range(row)]}
    return result

In [5]:
costs = np.array([[500, 550, 630, 1000, 800, 700],
                  [800, 700, 600, 950, 900, 930],
                  [1000, 960, 840, 650, 600, 700],
                  [1200, 1040, 980, 860, 880, 780]])
# 工厂最大产量
max_plant = [76, 88, 96, 40]
# 销售地需求量
max_cultivation = [42, 56, 44, 39, 60, 59]
# 使用线性规划
result = farming_program(costs, max_plant, max_cultivation)
# 输出结果
print(f'最大值为{result["objective"]}')
print('各变量的取值为：')
pprint(result['var'])

row:4 col:6
var= [[x00, x01, x02, x03, x04, x05], [x10, x11, x12, x13, x14, x15], [x20, x21, x22, x23, x24, x25], [x30, x31, x32, x33, x34, x35]]
flatten= <function trans_program.<locals>.<lambda> at 0x10d1cd430>
var[1] [x10, x11, x12, x13, x14, x15]
最大值为284230.0
各变量的取值为：
[[0.0, 0.0, 6.0, 39.0, 31.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 29.0, 59.0],
 [2.0, 56.0, 38.0, 0.0, 0.0, 0.0],
 [40.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
